In [1]:
import argparse
import os
import pickle
import numpy as np

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import distutils.errors
from hyperopt.pyll import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import mlflow
mlflow.__version__ #Answer to first question

'1.26.1'

# Pre-processing Steps

In [4]:
def dump_pickle(obj, filename):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [6]:
def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [7]:
def run(raw_data_path: str, dest_path: str, dataset: str = "green"):
    # load parquet files
    df_train = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2021-01.parquet")
    )
    df_valid = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2021-02.parquet")
    )
    df_test = read_dataframe(
        os.path.join(raw_data_path, f"{dataset}_tripdata_2021-03.parquet")
    )

    # extract the target
    target = 'duration'
    y_train = df_train[target].values
    y_valid = df_valid[target].values
    y_test = df_test[target].values

    # fit the dictvectorizer and preprocess data
    dv = DictVectorizer()
    X_train, dv = preprocess(df_train, dv, fit_dv=True)
    X_valid, _ = preprocess(df_valid, dv, fit_dv=False)
    X_test, _ = preprocess(df_test, dv, fit_dv=False)

    # create dest_path folder unless it already exists
    os.makedirs(dest_path, exist_ok=True)

    # save dictvectorizer and datasets
    dump_pickle(dv, os.path.join(dest_path, "dv.pkl"))
    dump_pickle((X_train, y_train), os.path.join(dest_path, "train.pkl"))
    dump_pickle((X_valid, y_valid), os.path.join(dest_path, "valid.pkl"))
    dump_pickle((X_test, y_test), os.path.join(dest_path, "test.pkl"))

In [8]:
run("./data/", "./data/assign2/")

In [9]:
lst_dir = os.listdir('./data/assign2')
print(len(lst_dir), lst_dir) # Answer to second question

4 ['valid.pkl', 'test.pkl', 'dv.pkl', 'train.pkl']


# Training Steps

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [11]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [12]:
os.getcwd()

'/home/ubuntu/mlops-zoomcamp/notebooks'

In [13]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Experiment-1")

<Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='Experiment-1', tags={}>

In [14]:
def run_training(data_path):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
    
    mlflow.sklearn.autolog(exclusive=False)
    
    with mlflow.start_run():
        mlflow.log_param("train-data-path", "/home/ubuntu/notebooks/data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/home/ubuntu/notebooks/data/green_tripdata_2021-02.parquet")
        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)

        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    
    print(rmse)
    mlflow.end_run()

In [15]:
run_training("./data/assign2")

6.729070933590364


In [16]:
mlflow.set_experiment("Optimization-1")

<Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='Optimization-1', tags={}>

In [17]:
def run_optimize(data_path, num_trials):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    def objective(params):

        with mlflow.start_run():
            
            mlflow.set_tag("model","RF_hyp")
            mlflow.log_params(params)
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_valid)
            rmse = mean_squared_error(y_valid, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)
            mlflow.sklearn.log_model(rf, artifact_path = "artifacts")

        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 10
    }

    rstate = np.random.default_rng(10)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )
    mlflow.end_run()


In [18]:
run_optimize("./data/assign2", 25)

  0%|                                                                                                                                | 0/25 [00:00<?, ?trial/s, best loss=?]

2022/05/31 04:15:04 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 882f8924c83c422d8cae48773028e6ac. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'auto\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fraction_leaf\', \'old_value\

  4%|████                                                                                                   | 1/25 [00:09<03:52,  9.70s/trial, best loss: 6.721531928622503]

2022/05/31 04:15:17 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID a5901d65af9b43e599f83a2ee0736641. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'auto\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fraction_leaf\', \'old_value\

  8%|████████▏                                                                                              | 2/25 [00:23<04:31, 11.82s/trial, best loss: 6.721531928622503]

...

 92%|████████████████████████████████████████████████████████████████████████████████████████████▉        | 23/25 [05:32<00:39, 19.76s/trial, best loss: 6.6257436134903935]

2022/05/31 04:21:07 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ce9766783d08465ab815fb035ea8f9b5. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'auto\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fraction_leaf\', \'old_value\

 96%|████████████████████████████████████████████████████████████████████████████████████████████████▉    | 24/25 [06:13<00:25, 25.96s/trial, best loss: 6.6257436134903935]

2022/05/31 04:21:36 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 5fb721c544ec4bd6873c59342f58c964. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'auto\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fraction_leaf\', \'old_value\

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [06:42<00:00, 16.09s/trial, best loss: 6.6257436134903935]


In [19]:
mlflow.set_experiment("Register_model-1")

<Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='Register_model-1', tags={}>

In [20]:
SPACE = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 10
}

In [21]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        params = space_eval(SPACE, params)
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # evaluate model on the validation and test sets
        valid_rmse = mean_squared_error(y_valid, rf.predict(X_valid), squared=False)
        mlflow.log_metric("valid_rmse", valid_rmse)
        test_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
        mlflow.log_metric("test_rmse", test_rmse)


def run_regiater_model(data_path, log_top):

    client = MlflowClient()

    # retrieve the top_n model runs and log the models to MLflow
    experiment = client.get_experiment_by_name("Optimization-1")
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=log_top,
        order_by=["metrics.rmse ASC"]
    )
    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # Identify model with the lowest test RMSE
    experiment = client.get_experiment_by_name("Register_model-1")
    best_run = client.search_runs(experiment_ids = experiment.experiment_id,
            run_view_type = ViewType.ACTIVE_ONLY,
            max_results=log_top,
            order_by=["metrics.test_rmse ASC"])[0]

    # register the best model
    run = f"runs/{best_run.info.run_id}/model"
    mlflow.register_model(model_uri = run, name = f"Registered model_{best_run.info.run_id}" )

In [22]:
run_regiater_model("./data/assign2", 5)

Successfully registered model 'Registered model_82695c6930c2493392922d91d4674b16'.
2022/05/31 04:24:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Registered model_82695c6930c2493392922d91d4674b16, version 1
Created version '1' of model 'Registered model_82695c6930c2493392922d91d4674b16'.
